# Ligthning Fast One-Shot Side Entanglement Optimization

## Imports

In [1]:
import numpy as np
import random
import itertools
import math
import time
import pickle
from typing import Optional, Tuple, cast, List, Dict
from bitarray.util import count_xor, urandom
from bitarray import frozenbitarray
from qiskit import Aer, QuantumCircuit, execute, QuantumRegister, ClassicalRegister
from qiskit.result.result import Result
from qiskit.aqua.components.optimizers import SLSQP, L_BFGS_B, CRS, DIRECT_L, DIRECT_L_RAND, ESCH, ISRES

## General auxiliary functions

In [557]:
""" save and load results to and from a file """
def save_results_to_disk(obj, name ):
    with open('results/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_results_from_file(name ):
    with open('results/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)

In [50]:
def get_combinations_n_etas_without_repeats(number_channels_to_discriminate: int = 2, eta_partitions: int = 20) -> List[List[float]]:
    """ from a given list of attenuations factors create a
        list of all combinatorial groups of n possible etas
        without repeats
        For us it is the same testing first eta 0.1 and second eta 0.2
        than first eta 0.2 and second eta 0.1
        Though, we will always put the greater value as the first group element
    """
    if number_channels_to_discriminate <= 1:
        raise ValueError('number_channels_to_discriminate must be at least 2')
    etas = np.append(np.arange(0, np.pi/2, np.pi/2/eta_partitions), np.pi/2)
    # get combinations of etas without repeats
    eta_groups = list(itertools.combinations(etas, number_channels_to_discriminate))
    return [sorted(eta_group, reverse=True) for eta_group in eta_groups]

In [51]:
result = get_combinations_n_etas_without_repeats(2, 20)

In [22]:
lst = random.sample(range(0, 1000), 10)

In [58]:
GLOBAL_ETA_GROUPS

[[0.07853981633974483, 0.0],
 [0.15707963267948966, 0.0],
 [0.23561944901923448, 0.0],
 [0.3141592653589793, 0.0],
 [0.39269908169872414, 0.0],
 [0.47123889803846897, 0.0],
 [0.5497787143782138, 0.0],
 [0.6283185307179586, 0.0],
 [0.7068583470577035, 0.0],
 [0.7853981633974483, 0.0],
 [0.8639379797371931, 0.0],
 [0.9424777960769379, 0.0],
 [1.0210176124166828, 0.0],
 [1.0995574287564276, 0.0],
 [1.1780972450961724, 0.0],
 [1.2566370614359172, 0.0],
 [1.335176877775662, 0.0],
 [1.413716694115407, 0.0],
 [1.4922565104551517, 0.0],
 [1.5707963267948966, 0.0],
 [0.15707963267948966, 0.07853981633974483],
 [0.23561944901923448, 0.07853981633974483],
 [0.3141592653589793, 0.07853981633974483],
 [0.39269908169872414, 0.07853981633974483],
 [0.47123889803846897, 0.07853981633974483],
 [0.5497787143782138, 0.07853981633974483],
 [0.6283185307179586, 0.07853981633974483],
 [0.7068583470577035, 0.07853981633974483],
 [0.7853981633974483, 0.07853981633974483],
 [0.8639379797371931, 0.0785398163397

In [59]:
GLOBAL_ETA_GROUP

[0, 0]

In [57]:
GLOBAL_BACKEND=Aer.get_backend('qasm_simulator')

optimization_setup = { 'optimizer_algorithms': ['CRS'],
                        'optimizer_iterations': [100],
                        'eta_partitions': 20, # number of partitions for the eta ranging from 0 to pi/2
                        'number_channels_to_discriminate': 2 }
GLOBAL_ETA_GROUPS = get_combinations_n_etas_without_repeats(optimization_setup['number_channels_to_discriminate'],
                                                           optimization_setup['eta_partitions'])
GLOBAL_ETA_GROUP = [0]*optimization_setup['number_channels_to_discriminate']
filename='20210412f_C2_A1_100_1000_2'


## Auxiliary functions for preparing the optimization (until the cost function)

In [562]:
def find_optimal_configurations(optimization_setup: dict, clone_setup: Optional[dict]=None) -> dict:
    """ Finds out the optimal configuration for each pair of attenuation levels
        using the configured optimization algorithm """
    eta_pairs_idx_to_optimize, optimal_configurations = _select_eta_pairs_to_optimize(clone_setup)

    print(f'number of eta_pairs_idx_to_optimize: {len(eta_pairs_idx_to_optimize)} -> {eta_pairs_idx_to_optimize}')

    program_start_time = time.time()
    print("Starting the execution")
    
    for eta_pair_idx in eta_pairs_idx_to_optimize:
        start_time = time.time()
        GLOBAL_ETA_PAIR = GLOBAL_ETA_PAIRS[eta_pair_idx]
        result = _compute_best_configuration(optimization_setup, GLOBAL_ETA_PAIR)
        optimal_configurations['probabilities'].append(result['best_probability'])
        optimal_configurations['configurations'].append(result['best_configuration'])
        optimal_configurations['best_algorithm'].append(result['best_algorithm'])
        optimal_configurations['number_calls_made'].append(result['number_calls_made'])
        end_time = time.time()
        print(f"Pair of etas # {eta_pair_idx} of {len(eta_pairs_idx_to_optimize)}, time taken this pair of etas: " +
              f'{np.round((end_time - start_time)/60, 0)} minutes' +
              f' and {np.round((end_time - start_time) % 60, 0)} seconds')
        print("total minutes taken this pair of etas: ", int(np.round((end_time - start_time) / 60)))
        print("total time taken so far: " +
              f'{np.round((end_time - program_start_time)/60, 0)} minutes' +
              f' and {np.round((end_time - program_start_time) % 60, 0)} seconds')
    end_time = time.time()
    print("total minutes of execution time: ", int(np.round((end_time - program_start_time) / 60)))
    print(f'Number eta pairs optimized: {len(eta_pairs_idx_to_optimize)}' +
          f'from the total eta pairs: {len(GLOBAL_ETA_PAIRS)} ')
    optimal_configurations['eta_pairs'] = GLOBAL_ETA_PAIRS

    return optimal_configurations

In [563]:
    def _select_eta_pairs_to_optimize(clone_setup: Optional[dict]) -> Tuple[List[int], dict]:
        """ from the given clone setup, select the eta pairs to be optimized,
            and set the non computed pairs configuration as default values   """

        eta_pair_idx_init, eta_pair_idx_end = _set_eta_pair_index_bounds(clone_setup)
        index_dict = _build_eta_pair_index_lists(eta_pair_idx_init, eta_pair_idx_end)
        default_optimal_configurations = _set_default_optimal_configurations(index_dict['eta_pair_idx_to_skip'])

        return (index_dict['eta_pair_idx_to_compute'],
                default_optimal_configurations)

In [564]:
def _set_default_optimal_configurations(eta_pair_idx_to_skip: List[int]) -> dict:
    """ Return the optimal configurations set to default values for the indexes to be skipped """
    elements_to_skip = len(eta_pair_idx_to_skip)

    configurations: List[dict] = []
    for eta_pair_idx in eta_pair_idx_to_skip:
        one_configuration = {'state_probability': 0,
                             'angle_rx1': 0,
                             'angle_ry1': 0,
                             'angle_rx0': 0,
                             'angle_ry0': 0,
                             'eta_pair': GLOBAL_ETA_PAIRS[eta_pair_idx]}
        configurations.append(one_configuration)

    return {'eta_pairs': [],
            'best_algorithm': ['NA'] * elements_to_skip,
            'probabilities': [0] * elements_to_skip,
            'configurations': configurations,
            'number_calls_made': [0] * elements_to_skip}

In [565]:
def _set_eta_pair_index_bounds(clone_setup: Optional[dict]) -> Tuple[int, int]:
    """ set the first and last eta pair index from which to optimize the configuration """
    total_eta_pairs = len(GLOBAL_ETA_PAIRS)

    if clone_setup is None or clone_setup['total_clones'] <= 1:
        return (0, total_eta_pairs)

    eta_pair_idx_init = int(np.floor(clone_setup['id_clone'] * total_eta_pairs / clone_setup['total_clones']))
    eta_pair_idx_end = min(int((clone_setup['id_clone'] + 1) *
                               total_eta_pairs / clone_setup['total_clones']), total_eta_pairs)
    return (eta_pair_idx_init, eta_pair_idx_end)

def _build_eta_pair_index_lists(eta_pair_idx_init: int, eta_pair_idx_end: int) -> Dict:
    """ create two lists with the the eta pair index to be computed and the index to be skipped """
    first_part_to_skip = list(range(0, eta_pair_idx_init))
    last_part_to_skip = list(range(eta_pair_idx_end, len(GLOBAL_ETA_PAIRS)))

    return {
        'eta_pair_idx_to_compute': list(range(eta_pair_idx_init, eta_pair_idx_end)),
        'eta_pair_idx_to_skip': first_part_to_skip + last_part_to_skip
    }

In [566]:
def _compute_best_configuration(optimization_setup: dict, GLOBAL_ETA_PAIR) -> dict:
    """ Find out the best configuration with a global pair of etas (channels) trying out
        a list of specified optimization algorithm """
    optimizer_algorithms = optimization_setup['optimizer_algorithms']
    optimizer_iterations = optimization_setup['optimizer_iterations']
    best_probability = 0
    best_configuration = []
    best_optimizer_algorithm = ""
    number_calls_made = 0

    for optimizer_algorithm, max_evals in zip(optimizer_algorithms, optimizer_iterations):
        print("Analyzing Optimizer Algorithm: ", optimizer_algorithm)
        if optimizer_algorithm == 'SLSQP':
            optimizer = SLSQP(maxiter=max_evals)
        if optimizer_algorithm == 'L_BFGS_B':
            optimizer = L_BFGS_B(maxfun=max_evals, maxiter=max_evals)
        if optimizer_algorithm == 'CRS':
            optimizer = CRS(max_evals=max_evals)
        if optimizer_algorithm == 'DIRECT_L':
            optimizer = DIRECT_L(max_evals=max_evals)
        if optimizer_algorithm == 'DIRECT_L_RAND':
            optimizer = DIRECT_L_RAND(max_evals=max_evals)
        if optimizer_algorithm == 'ESCH':
            optimizer = ESCH(max_evals=max_evals)
        if optimizer_algorithm == 'ISRES':
            optimizer = ISRES(max_evals=max_evals)

        ret = optimizer.optimize(num_vars=len(optimization_setup['initial_parameters']),
                                 objective_function=_cost_function,
                                 variable_bounds=optimization_setup['variable_bounds'],
                                 initial_point=optimization_setup['initial_parameters'])
#        print('Parámetros')
#        print(ret[0])
        print("Best Average Probability:", -ret[1])
        if (-ret[1]) > best_probability:
            best_configuration = ret[0]
            best_probability = -ret[1]
            number_calls_made = ret[2]
            best_optimizer_algorithm = optimizer_algorithm

    # Print results
    print("Final Best Optimizer Algorithm: ", best_optimizer_algorithm)
    print("Final Best Average Probability:", best_probability)
    print("Number of cost function calls made:", number_calls_made)
    print("Parameters Found: state_probability = " + str(best_configuration[0]) +
          ", " + u"\u03D5" + "rx1 = " + str(int(math.degrees(best_configuration[1]))) + u"\u00B0" +
          ", " + u"\u03D5" + "ry1 = " + str(int(math.degrees(best_configuration[2]))) + u"\u00B0" +
          ", " + u"\u03D5" + "rx0 = " + str(int(math.degrees(best_configuration[3]))) + u"\u00B0" +
          ", " + u"\u03D5" + "ry0 = " + str(int(math.degrees(best_configuration[4]))) + u"\u00B0" +
          ", " + u"\u03B7" + u"\u2080" + " = " + str(int(math.degrees(GLOBAL_ETA_PAIR[0]))) + u"\u00B0" +
          ", " + u"\u03B7" + u"\u2081" + " = " + str(int(math.degrees(GLOBAL_ETA_PAIR[1]))) + u"\u00B0")
    
    best_configuration_dict = {'state_probability': best_configuration[0],
                               'angle_rx1': best_configuration[1],
                               'angle_ry1': best_configuration[2],
                               'angle_rx0': best_configuration[3],
                               'angle_ry0': best_configuration[4],
                               'eta_pair': GLOBAL_ETA_PAIR} 
    
    return {'best_algorithm': best_optimizer_algorithm,
            'best_probability': best_probability,
            'best_configuration': best_configuration_dict,
            'number_calls_made': number_calls_made} 

In [567]:
def _cost_function(params: List[float]) -> float:
    """ Computes the cost of running a specific configuration for the number of plays
        defined in the optimization setup.
        Cost is computed as 1 (perfect probability) - average success probability for
        all the plays with the given configuration
        Returns the Cost (error probability).
    """
    configuration = {
        'state_probability': params[0],
        'angle_rx1': params[1],
        'angle_ry1': params[2],
        'angle_rx0': params[3],
        'angle_ry0': params[4],
        'eta_pair': GLOBAL_ETA_PAIR}

    return - compute_new_average_success_probability(configuration=configuration,
                                                       plays=GLOBAL_PLAYS)

## Auxiliary functions for fast compute the circuit and success average probability

In [568]:
def compute_new_average_success_probability(configuration: dict,
                                            plays: Optional[int] = 100,) -> float:
    """ Computes the average success probability of running a specific configuration
        for the number of plays defined in the configuration.
    """
    random_etas, eta_shots = _get_random_etas_and_eta_shots(plays)
    #guesses_eta = _run_all_circuits_and_return_guess(configuration, eta_shots)
    #return _check_guesses_and_return_average_success_probability(plays, random_etas, guesses_eta)
    return _run_all_circuits_and_return_cost(configuration, eta_shots)

In [569]:
def _get_random_etas_and_eta_shots(plays: int) -> Tuple[frozenbitarray, Tuple[int, int]]:
    """ create a random bit string with length the number of plays and calculate the
        number of shots for each eta based on the random string value (0 -> eta0, 1 -> eta1)
    """
    random_etas = frozenbitarray(urandom(plays))
    eta1_shots = random_etas.count()
    eta0_shots = plays - eta1_shots
    return (random_etas, (eta0_shots, eta1_shots))

In [570]:
def _check_guesses_and_return_average_success_probability(plays: int, 
                                                          random_etas: frozenbitarray, 
                                                          guesses_eta: Tuple[List[int],List[int]]) -> float:
    """ check the guessed etas with the initial random etas and compute the average with a xor bitwise operation """
    guesses = frozenbitarray([guesses_eta[random_eta].pop(0) for random_eta in random_etas])
    return 1 - (count_xor(random_etas, guesses) / plays)

In [571]:
def _run_all_circuits_and_return_cost(configuration: dict,
                                       eta_shots: Optional[Tuple[int, int]] = (1, 1)) -> Tuple[List[int],
                                                                                               List[int]]:
    """ Create a pair of Quantum Circuits, in its transpiled form, from a given configuration """
    if eta_shots is None:
        eta_shots = (1, 1)

    reordered_configuration = _reorder_configuration(configuration)
    eta_counts = [cast(Result, execute(_create_one_circuit(reordered_configuration, eta),
                                         backend=GLOBAL_BACKEND,
                                         shots=eta_shots[idx],
                                         memory=False).result()).get_counts()
                    for idx, eta in enumerate(reordered_configuration['eta_pair'])]
    
    counts_distribution=[0.0, 0.0, 0.0, 0.0]
    if '00' in eta_counts[0]: 
        counts_distribution[0]= eta_counts[0]['00']
    if '01' in eta_counts[0]: 
        counts_distribution[1]= eta_counts[0]['01']
    if '10' in eta_counts[0]: 
        counts_distribution[2]= eta_counts[0]['10']
    if '11' in eta_counts[0]: 
        counts_distribution[3]= eta_counts[0]['11']
    if '00' in eta_counts[1]: 
        counts_distribution[0]= max([counts_distribution[0], eta_counts[1]['00']])
    if '01' in eta_counts[1]: 
        counts_distribution[1]= max([counts_distribution[1], eta_counts[1]['01']])
    if '10' in eta_counts[1]: 
        counts_distribution[2]= max([counts_distribution[2], eta_counts[1]['10']])
    if '11' in eta_counts[1]: 
        counts_distribution[3]= max([counts_distribution[3], eta_counts[1]['11']])
#    print(counts_distribution)
#    print(configuration)

    cost= (counts_distribution[0] + counts_distribution[1] + counts_distribution[2] + counts_distribution[3])/GLOBAL_PLAYS
#    print(cost)
    return cost

In [572]:
def _reorder_configuration(configuration: dict):
    return {'state_probability': configuration['state_probability'],
            'angle_rx1': configuration['angle_rx1'],
            'angle_ry1': configuration['angle_ry1'],
            'angle_rx0': configuration['angle_rx0'],
            'angle_ry0': configuration['angle_ry0'],
            'eta_pair': reorder_pair(configuration['eta_pair'])
    }

In [573]:
def _create_one_circuit(configuration: dict,
                        eta: float) -> QuantumCircuit:
    """ Creates one circuit from a given configuration and eta """
    qreg_q = QuantumRegister(3, 'q')
    creg_c = ClassicalRegister(2, 'c')

    initial_state = _prepare_initial_state_entangled(configuration['state_probability'])

    circuit = QuantumCircuit(qreg_q, creg_c)
    circuit.initialize(initial_state, [0, 1])
    circuit.reset(qreg_q[2])
    circuit.cry(2 * eta, qreg_q[1], qreg_q[2])
    circuit.cx(qreg_q[2], qreg_q[1])
    circuit.barrier()
    circuit.rx(configuration['angle_rx1'], qreg_q[1])
    circuit.ry(configuration['angle_ry1'], qreg_q[1])
    circuit.rx(configuration['angle_rx0'], qreg_q[0])
    circuit.ry(configuration['angle_ry0'], qreg_q[0])
    circuit.cx(qreg_q[0], qreg_q[1])
    circuit.h(qreg_q[0])
    circuit.measure([0, 1], creg_c)
    return circuit

In [574]:
def _prepare_initial_state_entangled(state_probability: float) -> Tuple[complex, complex, complex, complex]:
    """ Prepare initial state: computing 'y' as the amplitudes  """
    # ORIGINAL
    return (0, np.sqrt(state_probability), np.sqrt(1 - state_probability), 0)
    # return (0, np.sqrt(1 - state_probability), np.sqrt(state_probability), 0)

In [575]:
def _get_counts_from_one_eta_counts(one_eta_counts: List[str]) -> List[int]:
    return [_guess_eta_from_counts(one_eta_count) for one_eta_count in one_eta_counts]

In [576]:
def _guess_eta_from_counts(counts: str) -> int:
    """ Decides which eta was used on the real execution from the 'counts' measured
        based on the guess strategy that is required to use
    """
    return _guess_eta_used_two_bit_strategy(counts)

In [577]:
def _guess_eta_used_two_bit_strategy(counts: str) -> int:
    """ Decides which eta was used on the real execution from the two 'counts' measured
        Qubits order MATTER!!!!
        "01" means that:
          the LEFTMOST bit (0) corresponds to the measurement of the qubit that goes THROUGH the channel
          and the RIGHTMOST bit (1) corresponds to the measurement of the qubit that goes OUTSIDE the channel
        Remember that we are only sending |01> + |10> entangles states
        Setting eta0 >= eta1:
            * outcome 00 -> eta0 as the most probable (more attenuation)
            * outcome 01 -> we do not know if there has been attenuation. 50% chance, random choice
            * outcome 10 -> eta1 as the most probable (less attenuation)
            * outcome 11 -> not possible, but in case we get it (from noisy simulation), 50% chance, random choice
    """
    if len(counts) != 2:
        raise ValueError('counts MUST be a two character length string')
    if counts == "00":
        return 0
    if counts == "10":
        return 1
    if counts == "01" or counts == "11":
        return random.choice([0, 1])
    raise ValueError("Accepted counts are '00', '01', '10', '11'")

## Launch Optimization

In [578]:
clone_setup=None
eta_pairs_idx_to_optimize, optimal_configurations = _select_eta_pairs_to_optimize(clone_setup)

print(f'number of eta_pairs_idx_to_optimize: {len(eta_pairs_idx_to_optimize)} -> {eta_pairs_idx_to_optimize}')

program_start_time = time.time()
print("Starting the execution")
    
for eta_pair_idx in eta_pairs_idx_to_optimize:
    start_time = time.time()
    GLOBAL_ETA_PAIR = GLOBAL_ETA_PAIRS[eta_pair_idx]
    result = _compute_best_configuration(optimization_setup, GLOBAL_ETA_PAIR)
    # print(result)
    optimal_configurations['probabilities'].append(result['best_probability'])
    optimal_configurations['configurations'].append(result['best_configuration'])
    optimal_configurations['best_algorithm'].append(result['best_algorithm'])
    optimal_configurations['number_calls_made'].append(result['number_calls_made'])
    end_time = time.time()
    print(f"Pair of etas # {eta_pair_idx+1} of {len(eta_pairs_idx_to_optimize)}, time taken this pair of etas: " +
              f'{np.round((end_time - start_time)/60, 0)} minutes' +
              f' and {np.round((end_time - start_time) % 60, 0)} seconds')
    print("total minutes taken this pair of etas: ", int(np.round((end_time - start_time) / 60)))
    print("total time taken so far: " +
              f'{np.round((end_time - program_start_time)/60, 0)} minutes' +
              f' and {np.round((end_time - program_start_time) % 60, 0)} seconds')
end_time = time.time()
print("total minutes of execution time: ", int(np.round((end_time - program_start_time) / 60)))
print(f'Number eta pairs optimized: {len(eta_pairs_idx_to_optimize)}' +
          f'from the total eta pairs: {len(GLOBAL_ETA_PAIRS)} ')
optimal_configurations['eta_pairs'] = GLOBAL_ETA_PAIRS
results= optimal_configurations

number of eta_pairs_idx_to_optimize: 210 -> [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209]
Starting the ex

Best Average Probability: 0.9752
Final Best Optimizer Algorithm:  CRS
Final Best Average Probability: 0.9752
Number of cost function calls made: 104
Parameters Found: state_probability = 0.0, ϕrx1 = 0°, ϕry1 = 0°, ϕrx0 = 0°, ϕry0 = 0°, η₀ = 76°, η₁ = 0°
Pair of etas # 17 of 210, time taken this pair of etas: 0.0 minutes and 4.0 seconds
total minutes taken this pair of etas:  0
total time taken so far: 1.0 minutes and 20.0 seconds
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.9898
Final Best Optimizer Algorithm:  CRS
Final Best Average Probability: 0.9898
Number of cost function calls made: 101
Parameters Found: state_probability = 0.0, ϕrx1 = 0°, ϕry1 = 0°, ϕrx0 = 0°, ϕry0 = 0°, η₀ = 81°, η₁ = 0°
Pair of etas # 18 of 210, time taken this pair of etas: 0.0 minutes and 4.0 seconds
total minutes taken this pair of etas:  0
total time taken so far: 1.0 minutes and 24.0 seconds
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.9978
Final Best Optimizer Algori

Best Average Probability: 0.9513
Final Best Optimizer Algorithm:  CRS
Final Best Average Probability: 0.9513
Number of cost function calls made: 117
Parameters Found: state_probability = 0.0, ϕrx1 = 0°, ϕry1 = 0°, ϕrx0 = 0°, ϕry0 = 0°, η₀ = 72°, η₁ = 4°
Pair of etas # 35 of 210, time taken this pair of etas: 0.0 minutes and 4.0 seconds
total minutes taken this pair of etas:  0
total time taken so far: 3.0 minutes and 48.0 seconds
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.9714
Final Best Optimizer Algorithm:  CRS
Final Best Average Probability: 0.9714
Number of cost function calls made: 102
Parameters Found: state_probability = 0.0, ϕrx1 = 0°, ϕry1 = 0°, ϕrx0 = 0°, ϕry0 = 0°, η₀ = 76°, η₁ = 4°
Pair of etas # 36 of 210, time taken this pair of etas: 0.0 minutes and 4.0 seconds
total minutes taken this pair of etas:  0
total time taken so far: 3.0 minutes and 52.0 seconds
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.9856
Final Best Optimizer Algori

Best Average Probability: 0.9428
Final Best Optimizer Algorithm:  CRS
Final Best Average Probability: 0.9428
Number of cost function calls made: 101
Parameters Found: state_probability = 0.0, ϕrx1 = 0°, ϕry1 = 0°, ϕrx0 = 0°, ϕry0 = 0°, η₀ = 72°, η₁ = 9°
Pair of etas # 53 of 210, time taken this pair of etas: 0.0 minutes and 4.0 seconds
total minutes taken this pair of etas:  0
total time taken so far: 4.0 minutes and 13.0 seconds
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.9634
Final Best Optimizer Algorithm:  CRS
Final Best Average Probability: 0.9634
Number of cost function calls made: 106
Parameters Found: state_probability = 0.0, ϕrx1 = 0°, ϕry1 = 0°, ϕrx0 = 0°, ϕry0 = 0°, η₀ = 76°, η₁ = 9°
Pair of etas # 54 of 210, time taken this pair of etas: 0.0 minutes and 4.0 seconds
total minutes taken this pair of etas:  0
total time taken so far: 4.0 minutes and 17.0 seconds
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.9783
Final Best Optimizer Algori

Best Average Probability: 0.946
Final Best Optimizer Algorithm:  CRS
Final Best Average Probability: 0.946
Number of cost function calls made: 102
Parameters Found: state_probability = 0.0, ϕrx1 = 0°, ϕry1 = 0°, ϕrx0 = 0°, ϕry0 = 0°, η₀ = 76°, η₁ = 13°
Pair of etas # 71 of 210, time taken this pair of etas: 0.0 minutes and 4.0 seconds
total minutes taken this pair of etas:  0
total time taken so far: 6.0 minutes and 49.0 seconds
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.9632
Final Best Optimizer Algorithm:  CRS
Final Best Average Probability: 0.9632
Number of cost function calls made: 100
Parameters Found: state_probability = 0.0, ϕrx1 = 0°, ϕry1 = 0°, ϕrx0 = 0°, ϕry0 = 0°, η₀ = 81°, η₁ = 13°
Pair of etas # 72 of 210, time taken this pair of etas: 0.0 minutes and 3.0 seconds
total minutes taken this pair of etas:  0
total time taken so far: 6.0 minutes and 53.0 seconds
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.9728
Final Best Optimizer Algori

...

Best Average Probability: 0.7339
Final Best Optimizer Algorithm:  CRS
Final Best Average Probability: 0.7339
Number of cost function calls made: 100
Parameters Found: state_probability = 0.0, ϕrx1 = 0°, ϕry1 = 0°, ϕrx0 = 0°, ϕry0 = 0°, η₀ = 58°, η₁ = 31°
Pair of etas # 125 of 210, time taken this pair of etas: 0.0 minutes and 6.0 seconds
total minutes taken this pair of etas:  0
total time taken so far: 10.0 minutes and 13.0 seconds
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.7688
Final Best Optimizer Algorithm:  CRS
Final Best Average Probability: 0.7688
Number of cost function calls made: 100
Parameters Found: state_probability = 0.0, ϕrx1 = 0°, ϕry1 = 0°, ϕrx0 = 0°, ϕry0 = 0°, η₀ = 63°, η₁ = 31°
Pair of etas # 126 of 210, time taken this pair of etas: 0.0 minutes and 4.0 seconds
total minutes taken this pair of etas:  0
total time taken so far: 10.0 minutes and 17.0 seconds
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.7969
Final Best Optimizer 

Best Average Probability: 0.8262
Final Best Optimizer Algorithm:  CRS
Final Best Average Probability: 0.8262
Number of cost function calls made: 101
Parameters Found: state_probability = 0.0, ϕrx1 = 0°, ϕry1 = 0°, ϕrx0 = 0°, ϕry0 = 0°, η₀ = 85°, η₁ = 36°
Pair of etas # 143 of 210, time taken this pair of etas: 0.0 minutes and 7.0 seconds
total minutes taken this pair of etas:  0
total time taken so far: 11.0 minutes and 29.0 seconds
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.8335
Final Best Optimizer Algorithm:  CRS
Final Best Average Probability: 0.8335
Number of cost function calls made: 101
Parameters Found: state_probability = 0.0, ϕrx1 = 0°, ϕry1 = 0°, ϕrx0 = 0°, ϕry0 = 0°, η₀ = 90°, η₁ = 36°
Pair of etas # 144 of 210, time taken this pair of etas: 0.0 minutes and 6.0 seconds
total minutes taken this pair of etas:  0
total time taken so far: 12.0 minutes and 35.0 seconds
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.5524
Final Best Optimizer 

Best Average Probability: 0.7061
Final Best Optimizer Algorithm:  CRS
Final Best Average Probability: 0.7061
Number of cost function calls made: 101
Parameters Found: state_probability = 0.0, ϕrx1 = 0°, ϕry1 = 0°, ϕrx0 = 0°, ϕry0 = 0°, η₀ = 72°, η₁ = 45°
Pair of etas # 161 of 210, time taken this pair of etas: 0.0 minutes and 4.0 seconds
total minutes taken this pair of etas:  0
total time taken so far: 13.0 minutes and 49.0 seconds
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.7305
Final Best Optimizer Algorithm:  CRS
Final Best Average Probability: 0.7305
Number of cost function calls made: 104
Parameters Found: state_probability = 0.0, ϕrx1 = 0°, ϕry1 = 0°, ϕrx0 = 0°, ϕry0 = 0°, η₀ = 76°, η₁ = 45°
Pair of etas # 162 of 210, time taken this pair of etas: 0.0 minutes and 4.0 seconds
total minutes taken this pair of etas:  0
total time taken so far: 13.0 minutes and 52.0 seconds
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.7406
Final Best Optimizer 

Best Average Probability: 0.6544
Final Best Optimizer Algorithm:  CRS
Final Best Average Probability: 0.6544
Number of cost function calls made: 101
Parameters Found: state_probability = 0.0, ϕrx1 = 0°, ϕry1 = 0°, ϕrx0 = 0°, ϕry0 = 0°, η₀ = 76°, η₁ = 54°
Pair of etas # 179 of 210, time taken this pair of etas: 0.0 minutes and 4.0 seconds
total minutes taken this pair of etas:  0
total time taken so far: 14.0 minutes and 12.0 seconds
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.6698
Final Best Optimizer Algorithm:  CRS
Final Best Average Probability: 0.6698
Number of cost function calls made: 102
Parameters Found: state_probability = 0.0, ϕrx1 = 0°, ϕry1 = 0°, ϕrx0 = 0°, ϕry0 = 0°, η₀ = 81°, η₁ = 54°
Pair of etas # 180 of 210, time taken this pair of etas: 0.0 minutes and 4.0 seconds
total minutes taken this pair of etas:  0
total time taken so far: 14.0 minutes and 16.0 seconds
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.6803
Final Best Optimizer 

Best Average Probability: 0.5341
Final Best Optimizer Algorithm:  CRS
Final Best Average Probability: 0.5341
Number of cost function calls made: 101
Parameters Found: state_probability = 0.17316610807508748, ϕrx1 = 69°, ϕry1 = 53°, ϕrx0 = 81°, ϕry0 = 68°, η₀ = 72°, η₁ = 67°
Pair of etas # 196 of 210, time taken this pair of etas: 0.0 minutes and 4.0 seconds
total minutes taken this pair of etas:  0
total time taken so far: 15.0 minutes and 30.0 seconds
Analyzing Optimizer Algorithm:  CRS
Best Average Probability: 0.561
Final Best Optimizer Algorithm:  CRS
Final Best Average Probability: 0.561
Number of cost function calls made: 101
Parameters Found: state_probability = 0.2804569627220101, ϕrx1 = 180°, ϕry1 = 180°, ϕrx0 = 0°, ϕry0 = 180°, η₀ = 76°, η₁ = 67°
Pair of etas # 197 of 210, time taken this pair of etas: 0.0 minutes and 4.0 seconds
total minutes taken this pair of etas:  0
total time taken so far: 16.0 minutes and 34.0 seconds
Analyzing Optimizer Algorithm:  CRS
Best Average Pr

In [583]:
results

{'eta_pairs': [(0.07853981633974483, 0.0),
  (0.15707963267948966, 0.0),
  (0.23561944901923448, 0.0),
  (0.3141592653589793, 0.0),
  (0.39269908169872414, 0.0),
  (0.47123889803846897, 0.0),
  (0.5497787143782138, 0.0),
  (0.6283185307179586, 0.0),
  (0.7068583470577035, 0.0),
  (0.7853981633974483, 0.0),
  (0.8639379797371931, 0.0),
  (0.9424777960769379, 0.0),
  (1.0210176124166828, 0.0),
  (1.0995574287564276, 0.0),
  (1.1780972450961724, 0.0),
  (1.2566370614359172, 0.0),
  (1.335176877775662, 0.0),
  (1.413716694115407, 0.0),
  (1.4922565104551517, 0.0),
  (1.5707963267948966, 0.0),
  (0.15707963267948966, 0.07853981633974483),
  (0.23561944901923448, 0.07853981633974483),
  (0.3141592653589793, 0.07853981633974483),
  (0.39269908169872414, 0.07853981633974483),
  (0.47123889803846897, 0.07853981633974483),
  (0.5497787143782138, 0.07853981633974483),
  (0.6283185307179586, 0.07853981633974483),
  (0.7068583470577035, 0.07853981633974483),
  (0.7853981633974483, 0.078539816339744

In [582]:
save_results_to_disk(results, filename)
# save_results_to_disk(results, '20210412d_C2_A1_100_10000')

In [584]:
qreg_q = QuantumRegister(3, 'q')
creg_c = ClassicalRegister(2, 'c')
circuit = QuantumCircuit(qreg_q, creg_c)
circuit.initialize([0,1,0,0], [0, 1])
circuit.reset(qreg_q[2])
circuit.cry(0, qreg_q[1], qreg_q[2])
circuit.cx(qreg_q[2], qreg_q[1])
circuit.barrier()
circuit.rx(0, qreg_q[1])
circuit.ry(0, qreg_q[1])
circuit.rx(0, qreg_q[0])
circuit.ry(0, qreg_q[0])
circuit.cx(qreg_q[0], qreg_q[1])
circuit.h(qreg_q[0])
circuit.measure([0, 1], creg_c)
circuit.draw()

┌──────────────────────┐               ░ ┌───────┐┌───────┐     ┌───┐┌─┐
q_0: ┤0                     ├───────────────░─┤ RX(0) ├┤ RY(0) ├──■──┤ H ├┤M├
     │  initialize(0,1,0,0) │         ┌───┐ ░ ├───────┤├───────┤┌─┴─┐└┬─┬┘└╥┘
q_1: ┤1                     ├────■────┤ X ├─░─┤ RX(0) ├┤ RY(0) ├┤ X ├─┤M├──╫─
     └──────────────────────┘┌───┴───┐└─┬─┘ ░ └───────┘└───────┘└───┘ └╥┘  ║ 
q_2: ──────────|0>───────────┤ RY(0) ├──■───░──────────────────────────╫───╫─
                             └───────┘      ░                          ║   ║ 
c: 2/══════════════════════════════════════════════════════════════════╩═══╩═
                                                                       1   0